In [5]:
#export
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [6]:
mpl.rcParams['image.cmap'] = 'gray'

In [7]:
# path = datasets.download_data(MNIST_URL, ext='.gz'); path

In [8]:
# with gzip.open(path, 'rb') as f:
#     ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

In [9]:
# x_train, y_train, x_valid, y_valid = map(tensor, (x_train,y_train,x_valid,y_valid))
# n, n_classes = x_train.shape

In [10]:
# x_train, x_train.shape, y_train, y_train.shape, y_train.min(), y_train.max()

In [11]:
# img = x_train[0] # get one imge
# label = y_train[0]
# img.type()

In [12]:
# plt.title(label.numpy())
# plt.imshow(img.view(28,28))

## Matrix mulitply (matmul)
The operation of a a matmul is essentially a dot product of each row by each column

Given two matrices A and B of size n x m. You can only do a matrix multiplication of number of columns in A == number of rows in B.

The new shape will be A number of rows by B number of columns


AxB

A = (n1,m1)

B = (n2,m2)

assert m1 == n2

new shape: (m1,n2)


In [13]:
a = np.random.random((20, 15))
b = np.random.random((15, 25))
(a@b).shape

(20, 25)

## TODO

* add links/imgs from deeplearning books

[Kahhn Academy Video](https://www.khanacademy.org/math/precalculus/x9e81a4f98389efdf:matrices/x9e81a4f98389efdf:multiplying-matrices-by-matrices/v/multiplying-a-matrix-by-a-matrix)

[Example](http://matrixmultiplication.xyz/)

In [14]:
weight = torch.randn(784, 10)
bias = torch.zeros(10)

In [15]:
def matmul(a, b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac == br
    result = torch.zeros(ar,bc)
    for i in range(ac):
           c[i]   = (a[i].unsqueeze(-1) * b).sum(dim=0) # why does this make sense
        
    

[computational linear algebra course](https://github.com/fastai/numerical-linear-algebra)

continue at 1:25

## Relu and Initialising 

In [16]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))



### Normalising data to have mean of 0 and standard deviation of 1

**NB** Only normalise data based on training mean and std

In [1]:
def normalize(data, mean, std): 
    return (data-mean)/std

In [18]:
x_train, y_train, x_valid, y_valid = get_data()

In [19]:
train_mean = x_train.mean()
train_std = x_train.std()

In [20]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

### Check is data is normalised

In [21]:
x_train.std(), x_train.mean()

(tensor(1.), tensor(-6.2598e-06))

In [22]:
num_samples, input_size = x_train.shape
num_classes = y_train.max()+1
num_samples, input_size, num_classes

(50000, 784, tensor(10))

### Basic architecture

Why is normalisation important?

**Papers**: 

[Fixup Initialization](https://arxiv.org/pdf/1901.09321.pdf)

[Understanding the difficulty of training deep feedforward neural networks](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)

Read 2.2 until backprob [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852)

In [23]:
num_hidden = 50

Dividing weight by square rrot of input activations helps normalise. It is a simplified kaming init 

In [24]:
w1 = torch.randn(input_size, num_hidden)/math.sqrt(input_size) # weights 1
b1 = torch.zeros(num_hidden) # bias 1
w2 = torch.randn(num_hidden, 1)/math.sqrt(num_hidden) # weights 2
b2 = torch.zeros(1)

In [25]:
# Without kaming init
w = torch.randn(input_size, num_hidden)
w.mean(), w.std() # This should be ~ (0,1) (mean,std)

(tensor(0.0020), tensor(0.9985))

In [26]:
# With kaming init
w = torch.randn(input_size, num_hidden)/math.sqrt(input_size)
w.mean(), w.std() # This should be ~ (0,1) (mean,std)

(tensor(8.3478e-05), tensor(0.0358))

### Creating a linear layer as shown in the image below with relu as the activation function

![Mathematical model of artificial neuron](https://www.researchgate.net/publication/320270458/figure/fig1/AS:551197154254848@1508427050805/Mathematical-model-of-artificial-neuron.png)

In [27]:
def lin(inputs, weights, bias): 
    return inputs@weights + bias # @ symbol is matrix multiplication operation
    

In [28]:
# with Kaiming
y_hat = lin(x_valid, w1, b1)
y_hat.mean(), y_hat.std()

(tensor(0.0212), tensor(0.9325))

In [29]:
# without Kaiming
y_hat = lin(x_valid, w, b1)
y_hat.mean(), y_hat.std()

(tensor(0.0530), tensor(0.9689))

In [30]:
# Our activation function
def relu(x): 
    return x.clamp_min(0.) - 0.5

In [31]:
# Kaiming
y_hat = relu(lin(x_valid, w1, b1))
y_hat.mean(), y_hat.std()

(tensor(-0.1228), tensor(0.5550))

We lose our mean and std because of the relu activation functions because we ar getting rid of all the negative numbers.

How do we solve, enter **Delving Deep into Rectifiers**

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [32]:
# kaiming init / he init for relu
w1 = torch.randn(input_size, num_hidden)*math.sqrt(2/input_size)
w1.mean(), w1.std()

(tensor(-0.0001), tensor(0.0507))

In [33]:
# Updated init
y_hat = relu(lin(x_valid, w1, b1))
y_hat.mean(), y_hat.std()

(tensor(-0.0454), tensor(0.7345))

got up to 1:43

## Forward Pass

In [34]:
w1 = torch.randn(input_size, num_hidden)/math.sqrt(2/input_size) # weights 1
b1 = torch.zeros(num_hidden) # bias 1
w2 = torch.randn(num_hidden, 1)/math.sqrt(2/num_hidden) # weights 2
b2 = torch.zeros(1)

In [35]:
def model(xb):
    lin1 = lin(xb, w1, b1)
    lin2  = relu(lin1)
    lin3 = lin(lin2, w2, b2)
    return lin3

In [36]:
x_valid.shape

torch.Size([10000, 784])

## Loss function - MSE (mean sqaured error)

In [60]:
model(x_valid).shape

torch.Size([10000, 1])

![mse formula](https://wikimedia.org/api/rest_v1/media/math/render/svg/e258221518869aa1c6561bb75b99476c4734108e)

In [61]:
def mse(output, target):
    return (output.squeeze(-1) - target).pow(2).mean()

In [62]:
# convert to float because we are using mse for loss (regression)
y_train, y_valid =  y_train.float(),  y_valid.float()

In [63]:
preds = model(x_valid)

In [64]:
mse(preds, y_valid)

tensor(1.1476e+08)

## Gradients and backward pass

1:48

[Chain Rule](https://www.khanacademy.org/math/ap-calculus-ab/ab-differentiation-2-new/ab-3-1a/v/chain-rule-introduction)

Matrix calaulus for deeplearning

continue from 1:58